In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['ieee-fraud-detection', 'data-preprocess']


In [2]:
import pickle
pickle_in = open("../input/data-preprocess/X_train.pickle","rb")
X_train = pickle.load(pickle_in).fillna(-999)
pickle_in = open("../input/data-preprocess/X_test.pickle","rb")
X_test = pickle.load(pickle_in).fillna(-999)
pickle_in = open("../input/data-preprocess/y_train.pickle","rb")
y_train = pickle.load(pickle_in)

In [3]:
from sklearn.decomposition import PCA
tf = PCA(n_components = 100)
tf.fit(pd.concat([X_train, X_test], 0, ignore_index = True))

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [4]:
X_train = tf.transform(X_train)
X_test = tf.transform(X_test)

In [5]:
del tf

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.svm import NuSVC
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from vecstack import stacking 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

/opt/conda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [7]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
def auc(y_true, y_pred):
    """ROC AUC metric for both binary and multiclass classification.
    
    Parameters
    ----------
    y_true : 1d numpy array
        True class labels
    y_pred : 2d numpy array
        Predicted probabilities for each class
    """
    ohe = OneHotEncoder(sparse=False)
    y_true = ohe.fit_transform(y_true.reshape(-1, 1))
    auc_score = roc_auc_score(y_true, y_pred)
    return auc_score


In [8]:
models = [ 
    GaussianNB(),
    LogisticRegression(random_state=42, n_jobs=-1),
    ExtraTreesClassifier(random_state=42, n_jobs=-1),
    AdaBoostClassifier(random_state=42, n_jobs=-1),
    RandomForestClassifier(random_state=42, n_jobs=-1),
    KNeighborsClassifier(n_jobs=-1),
    MLPClassifier(random_state=42),
    XGBClassifier(random_state=42),
    LGBMClassifier(random_state=42)
    ]
                  
print("Level 2 started")

S_tr, S_te = stacking(models,                     # list of models
                           X_train, y_train, X_test,   # data
                           regression=False,           # classification task (if you need 
                                                       #     regression - set to True)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and vote
                           needs_proba=True,          # predict class labels (if you need 
                                                       #     probabilities - set to True) 
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=auc,                 # metric: callable
                           n_folds=10,                  # number of folds
                           stratified=True,            # stratified split for folds
                           shuffle=True,               # shuffle the data
                           random_state=42,             # ensure reproducibility
                           verbose=1)                  # print all info

TypeError: __init__() got an unexpected keyword argument 'n_jobs'

In [9]:
oof_lrr = np.zeros(len(X_train)) 
pred_te_lrr = np.zeros(len(X_test))
skf = StratifiedKFold(n_splits=10, random_state=42)
for train_index, test_index in skf.split(S_tr, y_train):
    lrr = LogisticRegression()
    lrr.fit(S_tr[train_index],y_train[train_index])
    oof_lrr[test_index] = lrr.predict_proba(S_tr[test_index,:])[:,1]
    pred_te_lrr += lrr.predict_proba(S_te)[:,1] / skf.n_splits
    
print('stack CV score =',round(roc_auc_score(y_train,oof_lrr),5))

NameError: name 'S_tr' is not defined

In [10]:
sample_submission = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv', index_col='TransactionID')
sample_submission['isFraud'] = pred_te_lrr
sample_submission.to_csv('simple_stacks.csv')